In [3]:
import os
from functools import partial

In [4]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, random_split, Subset
from IPython.display import clear_output
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [1]:
from lared_laughter.fusion.dataset.segmentation import SegmentationDataset
from lared_laughter.fusion.dataset import FatherDataset
from lared_laughter.audio.dataset import AudioLaughterExtractor
from lared_laughter.audio.audio_utils import featurize_melspec
from lared_laughter.audio.models.segmentation import get_pretrained_segmentation_model
from lared_laughter.accel.dataset import AccelExtractor
from lared_laughter.constants import dataset_path, datasets_path, annot_exp_path
from lared_laughter.accel.models.resnet import SegmentationResnet
from lared_laughter.video.dataset.extractor import VideoExtractor
from lared_laughter.video.dataset.transforms import get_kinetics_val_transform
from lared_laughter.video.models.models import make_slow_pretrained_segmenter
from lared_laughter.utils import ious, plot_accel, plot_audio, load_examples

In [2]:
examples = load_examples(os.path.join(annot_exp_path, 'processed', 'examples_without_calibration.csv'))

In [10]:
examples[['intensity']].head(20)

,intensity
0,4.0
1,4.0
2,6.0
3,3.0
4,4.0
5,2.0
6,2.0
7,4.0
8,4.0
9,4.0


In [7]:
len(examples[examples['condition']=='audio'])

1318

In [11]:
len(examples[examples['condition']=='av'])

1318

In [6]:
extractors={}
# accel_ds_path = os.path.join(datasets_path, 'loose', 'accel_long.pkl')
# extractors['accel'] = {
#     'extractor': AccelExtractor(accel_ds_path, min_len=60), 
#     'id_column': 'hash'
# }

# audios_path = os.path.join(datasets_path, "loose", "lared_audios.pkl")
# extractors['audio'] = {
#     'extractor': AudioLaughterExtractor(audios_path, min_len=1.5, max_len=1.5), 
#     'id_column': 'hash'
# }

# videos_path = os.path.join(datasets_path, 'loose', 'video')
videos_path = '/home/jose/data/lared_video/video'
extractors['video'] = {
    'extractor': VideoExtractor(videos_path,
        transform=get_kinetics_val_transform(32, 256, True)), 
    'id_column': 'hash'
}

# annot_path = os.path.join(annot_exp_path, 'processed', 'continuous_corrected.pkl')
# ds = SegmentationDataset(examples, annot_path, extractors, 
#     window_len=90,
#     resize_len=60)
# raw_ds = SegmentationDataset(examples, annot_path, extractors)
ds = FatherDataset(examples, extractors, label_column='pressed_key', id_column='hash', )

In [7]:
videos_path

'/home/jose/data/lared_video/video'

In [8]:
for i in range(len(ds)):
    _ = ds[i]

func:video_from_path took: 0:00:00.002645
func:get_clip took: 0:00:00.040228
func:video_from_path took: 0:00:00.001091
func:get_clip took: 0:00:00.019568
func:video_from_path took: 0:00:00.001543
func:get_clip took: 0:00:00.035439
func:video_from_path took: 0:00:00.001057
func:get_clip took: 0:00:00.023257
func:video_from_path took: 0:00:00.001194
func:get_clip took: 0:00:00.016486
func:video_from_path took: 0:00:00.001009
func:get_clip took: 0:00:00.016891
func:video_from_path took: 0:00:00.001258
func:get_clip took: 0:00:00.023816
func:video_from_path took: 0:00:00.001760
func:get_clip took: 0:00:00.013534
func:video_from_path took: 0:00:00.001004
func:get_clip took: 0:00:00.012305
func:video_from_path took: 0:00:00.001312
func:get_clip took: 0:00:00.016377
func:video_from_path took: 0:00:00.001133
func:get_clip took: 0:00:00.013850
func:video_from_path took: 0:00:00.001078
func:get_clip took: 0:00:00.016929
func:video_from_path took: 0:00:00.001324
func:get_clip took: 0:00:00.016470

KeyboardInterrupt: 